## Load data

In [1]:
import pandas as pd;
import numpy as np;

In [2]:
# Load dataset (replace with your actual data loading method)
survey_df = pd.read_csv("../../data/final_survey_table.csv", low_memory=False)

In [3]:
survey_df

,What is your age (# years)?,What is your gender?,In which country do you currently reside?,What is the highest level of formal education that you have attained or plan to attain within the next 2 years?,Select the title most similar to your current role (or most recent title if retired),What is the size of the company where you are employed?,Approximately how many individuals are responsible for data science workloads at your place of business?,Does your current employer incorporate machine learning methods into their business?,What is your current yearly compensation (approximate $USD)?,Approximately how much money have you spent on machine learning and/or cloud computing products at your work in the past 5 years?,What is the primary tool that you use at work or school to analyze data?,How long have you been writing code to analyze data (at work or at school)?,What programming language would you recommend an aspiring data scientist to learn first?,Have you ever used a TPU (tensor processing unit)?,For how many years have you used machine learning methods?,income_category
0,40-44,Male,Australia,Master’s degree,Other,Enterprise,20+,I do not know,"250,000-299,999","$10,000-$99,999",Local development environments,1-2 years,Python,Used TPU,2-3 years,T20
1,22-24,Male,India,Bachelor’s degree,Other,Small,0,No (we do not use ML methods),"4,000-4,999",$0 (USD),Local development environments,< 1 years,Python,Never,< 1 years,B40
2,22-24,Male,India,Master’s degree,Data Scientist,Medium,20+,We are exploring ML methods (and may one day p...,"10,000-14,999",$100-$999,Local development environments,3-5 years,Python,Used TPU,2-3 years,M40
3,22-24,Female,United States of America,Bachelor’s degree,Data Scientist,Enterprise,20+,"We recently started using ML methods (i.e., mo...","80,000-89,999",$0 (USD),Local development environments,3-5 years,Python,Used TPU,3-4 years,T20
4,55-59,Male,Netherlands,Master’s degree,Other,Small,1-2,We are exploring ML methods (and may one day p...,$0-999,$100-$999,Local development environments,5-10 years,Python,Never,< 1 years,B40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7794,25-29,Male,India,Master’s degree,Data Scientist,Small,1-2,"We recently started using ML methods (i.e., mo...","1,000-1,999",$100-$999,Local development environments,3-5 years,Python,Never,2-3 years,B40
7795,22-24,Female,Other,Bachelor’s degree,Other,Medium,1-2,We are exploring ML methods (and may one day p...,"5,000-7,499",$100-$999,Local development environments,1-2 years,Python,Never,1-2 years,M40
7796,25-29,Male,China,I prefer not to answer,Data Engineer,Mid-Large,5-9,"We recently started using ML methods (i.e., mo...","20,000-24,999",$100-$999,Local development environments,1-2 years,Python,Used TPU,1-2 years,M40
7797,25-29,Male,Australia,Bachelor’s degree,Other,Large,5-9,No (we do not use ML methods),"60,000-69,999","$10,000-$99,999",Local development environments,3-5 years,Python,Never,1-2 years,T20


In [4]:
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7799 entries, 0 to 7798
Data columns (total 16 columns):
 #   Column                                                                                                                             Non-Null Count  Dtype 
---  ------                                                                                                                             --------------  ----- 
 0   What is your age (# years)?                                                                                                        7799 non-null   object
 1   What is your gender?                                                                                                               7799 non-null   object
 2   In which country do you currently reside?                                                                                          7799 non-null   object
 3   What is the highest level of formal education that you have attained or plan to attain within

In [5]:
# Define the column mapping
column_mapping = {
    "What is your age (# years)?": "age",
    "What is your gender?": "gender",
    "In which country do you currently reside?": "country",
    "What is the highest level of formal education that you have attained or plan to attain within the next 2 years?": "education_level",
    "Select the title most similar to your current role (or most recent title if retired)": "job_title",
    "What is the size of the company where you are employed?": "company_size",
    "Approximately how many individuals are responsible for data science workloads at your place of business?": "data_science_team_size",
    "Does your current employer incorporate machine learning methods into their business?": "ml_incorporation",
    "What is your current yearly compensation (approximate $USD)?": "yearly_compensation",
    "Approximately how much money have you spent on machine learning and/or cloud computing products at your work in the past 5 years?": "ml_spending",
    "What is the primary tool that you use at work or school to analyze data?": "primary_tool",
    "How long have you been writing code to analyze data (at work or at school)?": "coding_experience",
    "What programming language would you recommend an aspiring data scientist to learn first?": "recommended_language",
    "Have you ever used a TPU (tensor processing unit)?": "used_tpu",
    "For how many years have you used machine learning methods?": "ml_experience",
    "income_category": "income_category"
}

# Rename the columns
survey_df.rename(columns=column_mapping, inplace=True)

In [6]:
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7799 entries, 0 to 7798
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   age                     7799 non-null   object
 1   gender                  7799 non-null   object
 2   country                 7799 non-null   object
 3   education_level         7799 non-null   object
 4   job_title               7799 non-null   object
 5   company_size            7799 non-null   object
 6   data_science_team_size  7799 non-null   object
 7   ml_incorporation        7799 non-null   object
 8   yearly_compensation     7799 non-null   object
 9   ml_spending             7799 non-null   object
 10  primary_tool            7799 non-null   object
 11  coding_experience       7799 non-null   object
 12  recommended_language    7799 non-null   object
 13  used_tpu                7799 non-null   object
 14  ml_experience           7799 non-null   object
 15  inco

### 4.3.2.1 Domain-Specific Features

In [7]:
# Define a helper function to convert ranges to midpoints for experience
def convert_years_to_midpoint(range_str):
    if isinstance(range_str, str) and '-' in range_str:
        # Handle ranges like "2-3 years"
        low, high = map(int, range_str.replace(' years', '').split('-'))
        return (low + high) / 2
    elif isinstance(range_str, str) and range_str.startswith('<'):
        # Handle "less than" cases like "< 1 years"
        return 0.5  # Assign midpoint for "< 1 years"
    elif isinstance(range_str, str) and range_str.startswith('>') or range_str.endswith('+ years'):
        # Handle "greater than" cases like "> 10 years" or "20+ years"
        range_str = range_str.replace('>', '').replace('+ years', '').strip()
        return float(range_str) + 1  # Assume 1 year beyond the given value
    elif isinstance(range_str, str):
        # Handle direct numeric values
        return float(range_str.replace(' years', '').strip())
    return np.nan

# Apply conversion
survey_df['ML_Experience_Midpoint'] = survey_df[
    'ml_experience'
].apply(convert_years_to_midpoint).round().astype(int)

survey_df['Coding_Experience_Midpoint'] = survey_df[
    'coding_experience'
].apply(convert_years_to_midpoint).round().astype(int)

# Calculate Interaction Feature: ML Experience to Coding Ratio
survey_df['Total_Experience'] = (
    survey_df['ML_Experience_Midpoint'] +
    survey_df['Coding_Experience_Midpoint']
).round().astype(int)


In [8]:
# Drop the unused features
survey_df.drop(columns=[
                        'ML_Experience_Midpoint', 'Coding_Experience_Midpoint',
                        'ml_experience', 'coding_experience'
                        ], inplace=True)


In [9]:
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7799 entries, 0 to 7798
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   age                     7799 non-null   object
 1   gender                  7799 non-null   object
 2   country                 7799 non-null   object
 3   education_level         7799 non-null   object
 4   job_title               7799 non-null   object
 5   company_size            7799 non-null   object
 6   data_science_team_size  7799 non-null   object
 7   ml_incorporation        7799 non-null   object
 8   yearly_compensation     7799 non-null   object
 9   ml_spending             7799 non-null   object
 10  primary_tool            7799 non-null   object
 11  recommended_language    7799 non-null   object
 12  used_tpu                7799 non-null   object
 13  income_category         7799 non-null   object
 14  Total_Experience        7799 non-null   int32 
dtypes: i

## Data Encoding

In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Function to dynamically map ordinal categories
def dynamic_ordinal_mapping(df, columns):
    """
    Dynamically maps ordinal values for specified columns in a dataframe.
    Automatically assigns integer values based on the order of unique categories.

    Args:
        df (pd.DataFrame): Input dataframe.
        columns (list): List of columns to apply ordinal mapping.

    Returns:
        pd.DataFrame: Dataframe with ordinal-encoded columns.
        dict: Dictionary with column names as keys and their value-to-ordinal mappings as values.
    """
    column_mappings = {}  # To store the mappings for each column

    for col in columns:
        # Extract unique categories and sort them logically
        unique_categories = sorted(df[col].unique())
        ordinal_mapping = {category: i for i, category in enumerate(unique_categories)}
        
        # Store the mapping for the current column
        column_mappings[col] = ordinal_mapping

        # Replace categories with ordinal values
        df[col] = df[col].map(ordinal_mapping)
        
    return column_mappings, df

# Apply to ordinal columns in your dataset
ordinal_columns = [
    "age",
    "data_science_team_size",
    "yearly_compensation",
    "education_level",
    "ml_spending"
]

ordinal_map, survey_df = dynamic_ordinal_mapping(survey_df, ordinal_columns)

# Step 3: One-Hot Encoding for Nominal Categorical Variables
# Justification: Nominal data has no inherent order (e.g., gender, country).

nominal_columns = [
    "gender",
    "country",
    "primary_tool",
    "job_title",
    "ml_incorporation",
    "primary_tool",
    "recommended_language",
    "income_category"
]

survey_df = pd.get_dummies(survey_df, columns=nominal_columns, drop_first=False)

# Step 4: Label Encoding for Binary Columns
# Justification: Label encoding is efficient for binary columns.

binary_columns = ["used_tpu"]

for col in binary_columns:
    le = LabelEncoder()
    survey_df[col] = le.fit_transform(survey_df[col])

# Final Memory Optimization: Convert category columns back to 'category' dtype
for col in survey_df.select_dtypes(include='category').columns:
    survey_df[col] = survey_df[col].astype('category')

# Display the final optimized dataframe
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7799 entries, 0 to 7798
Columns: 104 entries, age to income_category_T20
dtypes: bool(96), int32(2), int64(5), object(1)
memory usage: 1.1+ MB


In [11]:
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7799 entries, 0 to 7798
Columns: 104 entries, age to income_category_T20
dtypes: bool(96), int32(2), int64(5), object(1)
memory usage: 1.1+ MB


## Feature Engineering

### 4.3.2.2 Automated Feature Generation

In [12]:
# Identify and drop duplicate columns
survey_df = survey_df.loc[:, ~survey_df.columns.duplicated()]

fact_profile_table = survey_df[['age', 'gender_Male', 'education_level', 
                        'yearly_compensation', 'ml_spending', 'used_tpu', 
                        'Total_Experience']]

dim_demographics = survey_df[['age', 'gender_Male', 'gender_Female']].drop_duplicates().reset_index(drop=True)
dim_demographics['demo_id'] = dim_demographics.index + 1

dim_company_details = survey_df[['company_size', 'data_science_team_size']].drop_duplicates().reset_index(drop=True)
dim_company_details['company_id'] = dim_company_details.index + 1

dim_country = survey_df[[col for col in survey_df.columns if col.startswith('country_')]].drop_duplicates().reset_index(drop=True)
dim_country['country_id'] = dim_country.index + 1

dim_job_title = survey_df[[col for col in survey_df.columns if col.startswith('job_title_')]].drop_duplicates().reset_index(drop=True)
dim_job_title['job_title_id'] = dim_job_title.index + 1

dim_ml_incorporation = survey_df[[col for col in survey_df.columns if col.startswith('ml_incorporation_')]].drop_duplicates().reset_index(drop=True)
dim_ml_incorporation['ml_id'] = dim_ml_incorporation.index + 1

dim_primary_tool = survey_df[[col for col in survey_df.columns if col.startswith('primary_tool_')]].drop_duplicates().reset_index(drop=True)
dim_primary_tool['tool_id'] = dim_primary_tool.index + 1

dim_recommended_language = survey_df[[col for col in survey_df.columns if col.startswith('recommended_language_')]].drop_duplicates().reset_index(drop=True)
dim_recommended_language['lang_id'] = dim_recommended_language.index + 1

dim_income_category = survey_df[[col for col in survey_df.columns if col.startswith('income_category_')]].drop_duplicates().reset_index(drop=True)
dim_income_category['income_id'] = dim_income_category.index + 1

fact_profile_table = survey_df \
    .merge(dim_demographics, on=['age', 'gender_Male'], how='left') \
    .merge(dim_company_details, on=['company_size', 'data_science_team_size'], how='left') \
    .merge(dim_country, how='left') \
    .merge(dim_job_title, how='left') \
    .merge(dim_ml_incorporation, how='left') \
    .merge(dim_primary_tool, how='left') \
    .merge(dim_recommended_language, how='left') \
    .merge(dim_income_category, how='left')
# 'income_id', 
fact_profile_table = fact_profile_table[['demo_id', 'company_id', 'country_id', 
                                         'job_title_id', 'ml_id', 'tool_id', 
                                         'lang_id', 'income_id',
                                         'yearly_compensation', 'ml_spending', 'used_tpu', 
                                         'Total_Experience']]

In [13]:
fact_profile_table

,demo_id,company_id,country_id,job_title_id,ml_id,tool_id,lang_id,income_id,yearly_compensation,ml_spending,used_tpu,Total_Experience
0,1,1,1,1,1,1,1,1,11,2,1,4
1,2,2,2,1,2,1,1,2,15,0,0,0
2,2,3,2,2,3,1,1,3,2,3,1,6
3,3,1,3,2,4,1,1,1,22,0,1,8
4,4,4,4,1,3,1,1,2,0,3,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...
7794,9,4,2,2,4,1,1,2,1,3,0,6
7795,3,6,16,1,3,1,1,3,17,3,0,4
7796,9,18,46,9,4,1,1,3,8,3,1,4
7797,9,17,1,1,2,1,1,1,19,2,0,6


In [14]:
# Import the feature tools library for feature engineering
import featuretools as ft

# Create an EntitySet and add entities
es = ft.EntitySet("survey_data")

# Add entities to the EntitySet
es = es.add_dataframe(dataframe_name="profileTable", dataframe=fact_profile_table, index="profile_id", make_index=True)
es = es.add_dataframe(dataframe_name="demographics", dataframe=dim_demographics, index="demo_id")
es = es.add_dataframe(dataframe_name="company", dataframe=dim_company_details, index="company_id")
es = es.add_dataframe(dataframe_name="country", dataframe=dim_country, index="country_id")
es = es.add_dataframe(dataframe_name="jobTitle", dataframe=dim_job_title, index="job_title_id")
es = es.add_dataframe(dataframe_name="mlIncorporation", dataframe=dim_ml_incorporation, index="ml_id")
es = es.add_dataframe(dataframe_name="PrimaryTool", dataframe=dim_primary_tool, index="tool_id")
es = es.add_dataframe(dataframe_name="RecommendedLanguage", dataframe=dim_recommended_language, index="lang_id")
es = es.add_dataframe(dataframe_name="IncomeCategory", dataframe=dim_income_category, index="income_id")


c:\Users\Huawei\OneDrive - Universiti Malaya\Desktop\SEMESTER 7\WIE3007_Data-Mining\Group Project\data-mining-warehousing-wages-analysis\venv\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
c:\Users\Huawei\OneDrive - Universiti Malaya\Desktop\SEMESTER 7\WIE3007_Data-Mining\Group Project\data-mining-warehousing-wages-analysis\venv\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(


In [15]:
es

Entityset: survey_data
  DataFrames:
    profileTable [Rows: 7799, Columns: 13]
    demographics [Rows: 22, Columns: 4]
    company [Rows: 35, Columns: 3]
    country [Rows: 59, Columns: 60]
    jobTitle [Rows: 10, Columns: 11]
    mlIncorporation [Rows: 6, Columns: 7]
    PrimaryTool [Rows: 6, Columns: 7]
    RecommendedLanguage [Rows: 4, Columns: 5]
    IncomeCategory [Rows: 3, Columns: 4]
  Relationships:
    No relationships

In [16]:
# Define the relationships between the tables
relationship_demographics = ft.Relationship(es,
                                        parent_dataframe_name="demographics", parent_column_name="demo_id",
                                        child_dataframe_name="profileTable", child_column_name="demo_id")

relationship_company = ft.Relationship(es,
                                        parent_dataframe_name="company", parent_column_name="company_id",
                                        child_dataframe_name="profileTable", child_column_name="company_id")

relationship_country = ft.Relationship(es,
                                        parent_dataframe_name="country", parent_column_name="country_id",
                                        child_dataframe_name="profileTable", child_column_name="country_id")

relationship_job_title = ft.Relationship(es,
                                        parent_dataframe_name="jobTitle", parent_column_name="job_title_id",
                                        child_dataframe_name="profileTable", child_column_name="job_title_id")

relationship_ml_incorporation = ft.Relationship(es,
                                        parent_dataframe_name="mlIncorporation", parent_column_name="ml_id",
                                        child_dataframe_name="profileTable", child_column_name="ml_id")

relationship_primary_tool = ft.Relationship(es,
                                        parent_dataframe_name="PrimaryTool", parent_column_name="tool_id",
                                        child_dataframe_name="profileTable", child_column_name="tool_id")

relationship_recommended_language = ft.Relationship(es,
                                        parent_dataframe_name="RecommendedLanguage", parent_column_name="lang_id",
                                        child_dataframe_name="profileTable", child_column_name="lang_id")

relationship_income_category = ft.Relationship(es,
                                        parent_dataframe_name="IncomeCategory", parent_column_name="income_id",
                                        child_dataframe_name="profileTable", child_column_name="income_id")

In [17]:
# Add the relationships to the EntitySet
es = es.add_relationship(relationship=relationship_demographics)
es = es.add_relationship(relationship=relationship_company)
es = es.add_relationship(relationship=relationship_country)
es = es.add_relationship(relationship=relationship_job_title)
es = es.add_relationship(relationship=relationship_ml_incorporation)
es = es.add_relationship(relationship=relationship_primary_tool)
es = es.add_relationship(relationship=relationship_recommended_language)
es = es.add_relationship(relationship=relationship_income_category)

In [18]:
es

Entityset: survey_data
  DataFrames:
    profileTable [Rows: 7799, Columns: 13]
    demographics [Rows: 22, Columns: 4]
    company [Rows: 35, Columns: 3]
    country [Rows: 59, Columns: 60]
    jobTitle [Rows: 10, Columns: 11]
    mlIncorporation [Rows: 6, Columns: 7]
    PrimaryTool [Rows: 6, Columns: 7]
    RecommendedLanguage [Rows: 4, Columns: 5]
    IncomeCategory [Rows: 3, Columns: 4]
  Relationships:
    profileTable.demo_id -> demographics.demo_id
    profileTable.company_id -> company.company_id
    profileTable.country_id -> country.country_id
    profileTable.job_title_id -> jobTitle.job_title_id
    profileTable.ml_id -> mlIncorporation.ml_id
    profileTable.tool_id -> PrimaryTool.tool_id
    profileTable.lang_id -> RecommendedLanguage.lang_id
    profileTable.income_id -> IncomeCategory.income_id

In [19]:
# Perform Deep Feature Synthesis
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="profileTable",  # Use the name of the target dataframe in the EntitySet
    agg_primitives=["mean", "sum", "count", ],  # Aggregation functions
    # trans_primitives=["multiply_numeric", "divide_numeric"],  # Transformation functions
    max_depth=2,
    n_jobs=3,  # Parallel processing
    verbose=True
)


Built 177 features
EntitySet scattered to 3 workers in 7 seconds
Elapsed: 00:01 | Progress: 100%|██████████


In [20]:
feature_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7799 entries, 0 to 7798
Columns: 177 entries, demo_id to IncomeCategory.SUM(profileTable.yearly_compensation)
dtypes: Int64(8), bool(90), category(1), float64(64), int64(14)
memory usage: 5.9 MB


In [21]:
def display_tb (feature_matrix_num = 0, first_n = 5):
  feature_matrixes = [feature_matrix]
  name = ["Survey_data"]

  print(f"Displaying \"{name[feature_matrix_num]}\"")

  print("____________________________________________________________________________")
  print("All suggested columns")
  for n in list(feature_matrixes[feature_matrix_num].columns):
    print(n)

  n=first_n
  print("____________________________________________________________________________")
  print(f"\n\nLet's examine the feature matrix generated")
  display(feature_matrixes[feature_matrix_num])

In [22]:
display_tb()

Displaying "Survey_data"
____________________________________________________________________________
All suggested columns
demo_id
company_id
country_id
job_title_id
ml_id
tool_id
lang_id
income_id
yearly_compensation
ml_spending
used_tpu
Total_Experience
demographics.age
demographics.gender_Male
demographics.gender_Female
company.company_size
company.data_science_team_size
country.country_Algeria
country.country_Argentina
country.country_Australia
country.country_Austria
country.country_Bangladesh
country.country_Belarus
country.country_Belgium
country.country_Brazil
country.country_Canada
country.country_Chile
country.country_China
country.country_Colombia
country.country_Czech Republic
country.country_Denmark
country.country_Egypt
country.country_France
country.country_Germany
country.country_Greece
country.country_Hong Kong (S.A.R.)
country.country_Hungary
country.country_India
country.country_Indonesia
country.country_Iran, Islamic Republic of...
country.country_Ireland
country.c

,demo_id,company_id,country_id,job_title_id,ml_id,tool_id,lang_id,income_id,yearly_compensation,ml_spending,...,RecommendedLanguage.SUM(profileTable.yearly_compensation),IncomeCategory.COUNT(profileTable),IncomeCategory.MEAN(profileTable.Total_Experience),IncomeCategory.MEAN(profileTable.ml_spending),IncomeCategory.MEAN(profileTable.used_tpu),IncomeCategory.MEAN(profileTable.yearly_compensation),IncomeCategory.SUM(profileTable.Total_Experience),IncomeCategory.SUM(profileTable.ml_spending),IncomeCategory.SUM(profileTable.used_tpu),IncomeCategory.SUM(profileTable.yearly_compensation)
profile_id,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,11,2,...,65130.0,3481,11.603275,2.555300,0.207699,13.982476,40391.0,8895.0,723.0,48673.0
1,2,2,2,1,2,1,1,2,15,0,...,65130.0,1728,4.744792,1.666667,0.239005,3.894097,8199.0,2880.0,413.0,6729.0
2,2,3,2,2,3,1,1,3,2,3,...,65130.0,2590,6.110425,2.068726,0.228185,9.869498,15826.0,5358.0,591.0,25562.0
3,3,1,3,2,4,1,1,1,22,0,...,65130.0,3481,11.603275,2.555300,0.207699,13.982476,40391.0,8895.0,723.0,48673.0
4,4,4,4,1,3,1,1,2,0,3,...,65130.0,1728,4.744792,1.666667,0.239005,3.894097,8199.0,2880.0,413.0,6729.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7794,9,4,2,2,4,1,1,2,1,3,...,65130.0,1728,4.744792,1.666667,0.239005,3.894097,8199.0,2880.0,413.0,6729.0
7795,3,6,16,1,3,1,1,3,17,3,...,65130.0,2590,6.110425,2.068726,0.228185,9.869498,15826.0,5358.0,591.0,25562.0
7796,9,18,46,9,4,1,1,3,8,3,...,65130.0,2590,6.110425,2.068726,0.228185,9.869498,15826.0,5358.0,591.0,25562.0


In [23]:
feature_matrix

,demo_id,company_id,country_id,job_title_id,ml_id,tool_id,lang_id,income_id,yearly_compensation,ml_spending,...,RecommendedLanguage.SUM(profileTable.yearly_compensation),IncomeCategory.COUNT(profileTable),IncomeCategory.MEAN(profileTable.Total_Experience),IncomeCategory.MEAN(profileTable.ml_spending),IncomeCategory.MEAN(profileTable.used_tpu),IncomeCategory.MEAN(profileTable.yearly_compensation),IncomeCategory.SUM(profileTable.Total_Experience),IncomeCategory.SUM(profileTable.ml_spending),IncomeCategory.SUM(profileTable.used_tpu),IncomeCategory.SUM(profileTable.yearly_compensation)
profile_id,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,11,2,...,65130.0,3481,11.603275,2.555300,0.207699,13.982476,40391.0,8895.0,723.0,48673.0
1,2,2,2,1,2,1,1,2,15,0,...,65130.0,1728,4.744792,1.666667,0.239005,3.894097,8199.0,2880.0,413.0,6729.0
2,2,3,2,2,3,1,1,3,2,3,...,65130.0,2590,6.110425,2.068726,0.228185,9.869498,15826.0,5358.0,591.0,25562.0
3,3,1,3,2,4,1,1,1,22,0,...,65130.0,3481,11.603275,2.555300,0.207699,13.982476,40391.0,8895.0,723.0,48673.0
4,4,4,4,1,3,1,1,2,0,3,...,65130.0,1728,4.744792,1.666667,0.239005,3.894097,8199.0,2880.0,413.0,6729.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7794,9,4,2,2,4,1,1,2,1,3,...,65130.0,1728,4.744792,1.666667,0.239005,3.894097,8199.0,2880.0,413.0,6729.0
7795,3,6,16,1,3,1,1,3,17,3,...,65130.0,2590,6.110425,2.068726,0.228185,9.869498,15826.0,5358.0,591.0,25562.0
7796,9,18,46,9,4,1,1,3,8,3,...,65130.0,2590,6.110425,2.068726,0.228185,9.869498,15826.0,5358.0,591.0,25562.0


demographicsStats
<Feature: demographics.COUNT(profileTable)>
<Feature: demographics.MEAN(profileTable.yearly_compensation)>
<Feature: demographics.SUM(profileTable.yearly_compensation)>,

companyStats
<Feature: company.COUNT(profileTable)>,
<Feature: company.MEAN(profileTable.yearly_compensation)>,
<Feature: company.SUM(profileTable.yearly_compensation)>,

countryStats
<Feature: country.COUNT(profileTable)>
<Feature: country.MEAN(profileTable.yearly_compensation)>
<Feature: country.SUM(profileTable.yearly_compensation)>

jobTitleStats
<Feature: jobTitle.SUM(profileTable.yearly_compensation)>
<Feature: jobTitle.MEAN(profileTable.yearly_compensation)>
<Feature: jobTitle.SUM(profileTable.yearly_compensation)>

mlIncorporationStats
<Feature: mlIncorporation.SUM(profileTable.yearly_compensation)>
<Feature: mlIncorporation.MEAN(profileTable.yearly_compensation)>
<Feature: mlIncorporation.SUM(profileTable.yearly_compensation)>

primaryToolStats
<Feature: PrimaryTool.SUM(profileTable.yearly_compensation)>
<Feature: PrimaryTool.MEAN(profileTable.yearly_compensation)>
<Feature: PrimaryTool.SUM(profileTable.yearly_compensation)>

recommendedLanguageStats
<Feature: RecommendedLanguage.SUM(profileTable.yearly_compensation)>
<Feature: RecommendedLanguage.MEAN(profileTable.yearly_compensation)>
<Feature: RecommendedLanguage.SUM(profileTable.yearly_compensation)>

incomeCategoryStats
<Feature: IncomeCategory.SUM(profileTable.yearly_compensation)>
<Feature: IncomeCategory.MEAN(profileTable.yearly_compensation)>
<Feature: IncomeCategory.SUM(profileTable.yearly_compensation)>

In [24]:
# Reset index to merge with the fact_profile_table
feature_matrix_reset = feature_matrix.reset_index()

# Merge feature matrix with fact_profile_table
fact_profile_table_with_features = fact_profile_table.merge(
    feature_matrix_reset, 
    on='profile_id', 
    how='left'
)

# Filter for desired features
selected_features = [
    'demographics.COUNT(profileTable)',
    'demographics.MEAN(profileTable.yearly_compensation)',
    'demographics.SUM(profileTable.yearly_compensation)',
    'company.COUNT(profileTable)',
    'company.MEAN(profileTable.yearly_compensation)',
    'company.SUM(profileTable.yearly_compensation)',
    'country.COUNT(profileTable)',
    'country.MEAN(profileTable.yearly_compensation)',
    'country.SUM(profileTable.yearly_compensation)',
    'jobTitle.SUM(profileTable.yearly_compensation)',
    'jobTitle.MEAN(profileTable.yearly_compensation)',
    'jobTitle.SUM(profileTable.yearly_compensation)',
    'mlIncorporation.SUM(profileTable.yearly_compensation)',
    'mlIncorporation.MEAN(profileTable.yearly_compensation)',
    'PrimaryTool.SUM(profileTable.yearly_compensation)',
    'PrimaryTool.MEAN(profileTable.yearly_compensation)',
    'RecommendedLanguage.SUM(profileTable.yearly_compensation)',
    'RecommendedLanguage.MEAN(profileTable.yearly_compensation)',
    'IncomeCategory.SUM(profileTable.yearly_compensation)',
    'IncomeCategory.MEAN(profileTable.yearly_compensation)',
]

# Select only those features
feature_matrix_selected = feature_matrix_reset[['profile_id'] + selected_features]

# Merge with fact_profile_table
fact_profile_table_with_features = fact_profile_table.merge(
    feature_matrix_selected, 
    on='profile_id', 
    how='left'
)

fact_profile_table_with_features

,profile_id,demo_id,company_id,country_id,job_title_id,ml_id,tool_id,lang_id,income_id,yearly_compensation,...,jobTitle.MEAN(profileTable.yearly_compensation),jobTitle.SUM(profileTable.yearly_compensation),mlIncorporation.SUM(profileTable.yearly_compensation),mlIncorporation.MEAN(profileTable.yearly_compensation),PrimaryTool.SUM(profileTable.yearly_compensation),PrimaryTool.MEAN(profileTable.yearly_compensation),RecommendedLanguage.SUM(profileTable.yearly_compensation),RecommendedLanguage.MEAN(profileTable.yearly_compensation),IncomeCategory.SUM(profileTable.yearly_compensation),IncomeCategory.MEAN(profileTable.yearly_compensation)
0,0,1,1,1,1,1,1,1,1,11,...,10.768531,8281.0,4161.0,9.205752,49625.0,10.529387,65130.0,10.425804,48673.0,13.982476
1,1,2,2,2,1,2,1,1,2,15,...,10.768531,8281.0,9950.0,9.299065,49625.0,10.529387,65130.0,10.425804,6729.0,3.894097
2,2,2,3,2,2,3,1,1,3,2,...,10.486430,30138.0,16268.0,9.805907,49625.0,10.529387,65130.0,10.425804,25562.0,9.869498
3,3,3,1,3,2,4,1,1,1,22,...,10.486430,30138.0,20349.0,10.864389,49625.0,10.529387,65130.0,10.425804,48673.0,13.982476
4,4,4,4,4,1,3,1,1,2,0,...,10.768531,8281.0,16268.0,9.805907,49625.0,10.529387,65130.0,10.425804,6729.0,3.894097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7794,7794,9,4,2,2,4,1,1,2,1,...,10.486430,30138.0,20349.0,10.864389,49625.0,10.529387,65130.0,10.425804,6729.0,3.894097
7795,7795,3,6,16,1,3,1,1,3,17,...,10.768531,8281.0,16268.0,9.805907,49625.0,10.529387,65130.0,10.425804,25562.0,9.869498
7796,7796,9,18,46,9,4,1,1,3,8,...,10.570571,3520.0,20349.0,10.864389,49625.0,10.529387,65130.0,10.425804,25562.0,9.869498
7797,7797,9,17,1,1,2,1,1,1,19,...,10.768531,8281.0,9950.0,9.299065,49625.0,10.529387,65130.0,10.425804,48673.0,13.982476


## Dimensionality Reduction

In [25]:
# Merge fact table with dimensional tables
merged_df = fact_profile_table_with_features \
    .merge(dim_demographics, on="demo_id", how="left") \
    .merge(dim_company_details, on="company_id", how="left") \
    .merge(dim_country, on="country_id", how="left") \
    .merge(dim_job_title, on="job_title_id", how="left") \
    .merge(dim_ml_incorporation, on="ml_id", how="left") \
    .merge(dim_primary_tool, on="tool_id", how="left") \
    .merge(dim_recommended_language, on="lang_id", how="left") \
    .merge(dim_income_category, on="income_id", how="left")

# Drop unnecessary ID columns if they're not needed for PCA
merged_df = merged_df.drop(['demo_id', 'company_id', 'country_id', 'job_title_id', 
                            'ml_id', 'tool_id', 'lang_id', 'income_id'], axis=1)

# Display the resulting merged DataFrame
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7799 entries, 0 to 7798
Columns: 118 entries, profile_id to income_category_T20
dtypes: Int64(3), bool(90), category(1), float64(17), int64(7)
memory usage: 2.3 MB


In [26]:
merged_df

,profile_id,yearly_compensation,ml_spending,used_tpu,Total_Experience,demographics.COUNT(profileTable),demographics.MEAN(profileTable.yearly_compensation),demographics.SUM(profileTable.yearly_compensation),company.COUNT(profileTable),company.MEAN(profileTable.yearly_compensation),...,primary_tool_Cloud-based data software & APIs,primary_tool_Local development environments,primary_tool_Other,recommended_language_Other,recommended_language_Python,recommended_language_R,recommended_language_SQL,income_category_B40,income_category_M40,income_category_T20
0,0,11,2,1,4,638,11.463950,7314.0,1048,10.997137,...,False,True,False,False,True,False,False,False,False,True
1,1,15,0,0,0,902,8.657428,7809.0,352,6.738636,...,False,True,False,False,True,False,False,True,False,False
2,2,2,3,1,6,902,8.657428,7809.0,101,10.534653,...,False,True,False,False,True,False,False,False,True,False
3,3,22,0,1,8,159,7.880503,1253.0,1048,10.997137,...,False,True,False,False,True,False,False,False,False,True
4,4,0,3,0,8,158,11.063291,1748.0,986,8.873225,...,False,True,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7794,7794,1,3,0,6,1706,10.340563,17641.0,986,8.873225,...,False,True,False,False,True,False,False,True,False,False
7795,7795,17,3,0,4,159,7.880503,1253.0,321,10.367601,...,False,True,False,False,True,False,False,False,True,False
7796,7796,8,3,1,4,1706,10.340563,17641.0,197,10.532995,...,False,True,False,False,True,False,False,False,True,False
7797,7797,19,2,0,6,1706,10.340563,17641.0,231,10.917749,...,False,True,False,False,True,False,False,False,False,True


## Dimensionality Reduction

### 4.3.3.1 Remove Features with Poor Generalization
1. Drop or ignore dim_income_category as the income category B40, M40, T20 is only in Malaysia and it event didnt have same standard accross state in Malaysia.

In [27]:
dim_income_category

,income_category_B40,income_category_M40,income_category_T20,income_id
1,False,False,True,1
2,True,False,False,2
3,False,True,False,3


2. Remove duplicate column in dim_primary_tool

In [28]:
dim_primary_tool.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 1 to 6
Data columns (total 7 columns):
 #   Column                                         Non-Null Count  Dtype
---  ------                                         --------------  -----
 0   primary_tool_Advanced statistical software     6 non-null      bool 
 1   primary_tool_Basic statistical software        6 non-null      bool 
 2   primary_tool_Business intelligence software    6 non-null      bool 
 3   primary_tool_Cloud-based data software & APIs  6 non-null      bool 
 4   primary_tool_Local development environments    6 non-null      bool 
 5   primary_tool_Other                             6 non-null      bool 
 6   tool_id                                        6 non-null      int64
dtypes: bool(6), int64(1)
memory usage: 132.0 bytes


In [29]:
dim_primary_tool = dim_primary_tool.loc[:, ~dim_primary_tool.columns.duplicated()]
dim_primary_tool

,primary_tool_Advanced statistical software,primary_tool_Basic statistical software,primary_tool_Business intelligence software,primary_tool_Cloud-based data software & APIs,primary_tool_Local development environments,primary_tool_Other,tool_id
1,False,False,False,False,True,False,1
2,False,True,False,False,False,False,2
3,False,False,False,False,False,True,3
4,True,False,False,False,False,False,4
5,False,False,False,True,False,False,5
6,False,False,True,False,False,False,6


3. Map dim_ml_incorporation to oridinal value rank experience from low to high

In [30]:
# Define the ordinal mapping based on experience (low to high)
ordinal_mapping = {
    'ml_incorporation_No (we do not use ML methods)': 0,
    'ml_incorporation_We use ML methods for generating insights (but do not put working models into production)': 1,
    'ml_incorporation_We are exploring ML methods (and may one day put a model into production)': 2,
    'ml_incorporation_We recently started using ML methods (i.e., models in production for less than 2 years)': 3,
    'ml_incorporation_We have well established ML methods (i.e., models in production for more than 2 years)': 4,
}
ordinal_map["ml_experience"] = {key.replace('ml_incorporation_', ''): value for key, value in ordinal_mapping.items()}

# Apply the lambda function and assign the result to a new column
dim_ml_incorporation = dim_ml_incorporation.copy()
dim_ml_incorporation.loc[:, 'ml_experience'] = dim_ml_incorporation.apply(
    lambda x: max([ordinal_mapping[col] for col in dim_ml_incorporation.columns if x[col]] or [0]), axis=1
)

# Drop unused feature
dim_ml_incorporation = dim_ml_incorporation.drop(columns=ordinal_mapping.keys())
dim_ml_incorporation

KeyError: 'ml_incorporation_I do not know'

In [293]:
# Concat dim_ml_incorporation to fact_table
fact_table = pd.concat([fact_table, dim_ml_incorporation], axis=1)
fact_table

,age,gender_Male,education_level,yearly_compensation,ml_spending,used_tpu,Total_Experience,ml_experience
0,5,True,3,11,2,1,4,0
1,1,True,0,15,0,0,0,0
2,1,True,3,2,3,1,6,2
3,1,False,0,22,0,1,8,3
4,8,True,3,0,3,0,8,2
...,...,...,...,...,...,...,...,...
7794,2,True,3,1,3,0,6,3
7795,1,False,0,17,3,0,4,2
7796,2,True,2,8,3,1,4,3
7797,2,True,0,19,2,0,6,0


In [294]:
# Save the mapping between columns value to ordinal value
ordinal_map

{'age': {'18-21': 0,
  '22-24': 1,
  '25-29': 2,
  '30-34': 3,
  '35-39': 4,
  '40-44': 5,
  '45-49': 6,
  '50-54': 7,
  '55-59': 8,
  '60-69': 9,
  '70+': 10},
 'data_science_team_size': {'0': 0,
  '1-2': 1,
  '10-14': 2,
  '15-19': 3,
  '20+': 4,
  '3-4': 5,
  '5-9': 6},
 'yearly_compensation': {'$0-999': 0,
  '1,000-1,999': 1,
  '10,000-14,999': 2,
  '100,000-124,999': 3,
  '125,000-149,999': 4,
  '15,000-19,999': 5,
  '150,000-199,999': 6,
  '2,000-2,999': 7,
  '20,000-24,999': 8,
  '200,000-249,999': 9,
  '25,000-29,999': 10,
  '250,000-299,999': 11,
  '3,000-3,999': 12,
  '30,000-39,999': 13,
  '300,000-500,000': 14,
  '4,000-4,999': 15,
  '40,000-49,999': 16,
  '5,000-7,499': 17,
  '50,000-59,999': 18,
  '60,000-69,999': 19,
  '7,500-9,999': 20,
  '70,000-79,999': 21,
  '80,000-89,999': 22,
  '90,000-99,999': 23,
  '> $500,000': 24},
 'education_level': {'Bachelor’s degree': 0,
  'Doctoral degree': 1,
  'I prefer not to answer': 2,
  'Master’s degree': 3,
  'No formal education 